In [1]:
# I.Import working libraries
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
import glob 
import math

In [2]:
# II.Load all PLA csv file into one data files
all_files = glob.glob("/Users/thanhthaopro/Desktop/Working/SW/PLA Tag/*.csv")
#all_files

In [3]:
#II.1. Read all PLA inputs files into dataframe
#1.1. Creating empty SW list first. Then adding a for loop to look into all files line by line. Then append the empty list with all files
list_df = []
for i in all_files:
    df = pd.read_csv(i)
    list_df.append(df)

#1.2. check data len
len(list_df)

33

In [4]:
#2.Concat into one big file dataframe
df = pd.concat(list_df)
#Saving the dataframe
df = pd.DataFrame(df)
#Check if there is extra character such as '_3mmEE' in 'ProcessTag'
#df2=df[df['ProcessTag'].str.contains("P1276.4-2_3mmEE")]
#df2
#1.3. Removing P in ProcessTag and change the "ProcessTag" to "PLADot" then splitting DOT process with whatwever after "_"
df['PLADot']=df['ProcessTag'].str[1:]
#df[['PLADot','extraTag']] = df['PLADot'].str.split('_',expand=True)
#df['PLADot'] = df['PLADot'].str.split("_")[0]
#Create PLAFab column for items that cannot find the exact PLADot match
# Split the 'PLADot' column and create the 'PLAFab' column
df['PLAFab'] = df['PLADot'].str.split('.').str[0]

#1.4 Format data types - convert SRRDieArea_mm2 from float to integer then string since str function can only concat string values
df[["SRRDieArea_mm2"]] = df[["SRRDieArea_mm2"]].astype("int")
df[["SRRDieArea_mm2"]] = df[["SRRDieArea_mm2"]].astype("str")
#df[["PLADot"]] = df[["PLADot"]].astype("str")
#Insert 'PLADotDieArea' as the column name and the value is the concatcenate of "PLADot" and "SRRDieArea_mm2"
#df['PLADotDieArea'] = df['PLADot'] + '_' + df['SRRDieArea_mm2']
df['PLADotDieArea'] = df['PLADot'] + '_' + df['SRRDieArea_mm2']
# Create the 'PLAFabDieArea' column
df['PLAFabDieArea'] = df['PLAFab'] + '_' + df['SRRDieArea_mm2']
df

,AnalysisDate,ProcessTag,SRROGD_um,SRRPGD_um,SRRDeviceX_um,SRRDeviceY_um,SRRDieArea_mm2,AspectRatio,LFU,DPFOGD,...,FractureWindowOGD_um,FractureWindowPGD_um,GrossDPW,EdgeExclusionPrimary_mm,EdgeExclusionFull_mm,FrameBuildConfidenceLevel,PLADot,PLAFab,PLADotDieArea,PLAFabDieArea
0,24-May-24,P1222.4-1,2000.16,2000.88,2000.16,2000.88,4,0.999640,0.900433,15,...,31073.86,24862.42,15633,3,3,medium,1222.4-1,1222,1222.4-1_4,1222_4
1,24-May-24,P1222.4-1,2100.60,2000.88,2100.60,2000.88,4,1.049838,0.944090,15,...,32580.46,24862.42,14902,3,3,medium,1222.4-1,1222,1222.4-1_4,1222_4
2,24-May-24,P1222.4-1,2201.04,2000.88,2201.04,2000.88,4,1.100036,0.922027,14,...,31819.06,24862.42,14240,3,3,medium,1222.4-1,1222,1222.4-1_4,1222_4
3,24-May-24,P1222.4-1,2300.40,2000.88,2300.40,2000.88,4,1.149694,0.893736,13,...,30842.74,24862.42,13638,3,3,medium,1222.4-1,1222,1222.4-1_4,1222_4
4,24-May-24,P1222.4-1,2400.84,2000.88,2400.84,2000.88,4,1.199892,0.931572,13,...,32148.46,24862.42,13075,3,3,medium,1222.4-1,1222,1222.4-1_4,1222_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68361,13-Nov-23,P1280,25200.00,31200.48,25200.00,31200.48,786,0.807680,0.985427,1,...,25994.98,32525.38,66,5,5,NaN,1280,1280,1280_786,1280_786
68362,13-Nov-23,P1280,25200.00,31301.28,25200.00,31301.28,788,0.805079,0.988481,1,...,25994.98,32626.18,66,5,5,NaN,1280,1280,1280_788,1280_788
68363,13-Nov-23,P1280,25200.00,31400.64,25200.00,31400.64,791,0.802531,0.991492,1,...,25994.98,32725.54,65,5,5,NaN,1280,1280,1280_791,1280_791
68364,13-Nov-23,P1280,25200.00,31500.00,25200.00,31500.00,793,0.800000,0.994502,1,...,25994.98,32824.90,64,5,5,NaN,1280,1280,1280_793,1280_793


In [5]:
#dftest=df[df['ProcessTag'].str.contains("P1276")]
#dftest

In [6]:
#1.5. Saving the dataframe
df.to_csv("/Users/thanhthaopro/Desktop/Working/SW/PLA_Dot_SW_inputs.csv")
#dftest.to_csv("/Users/thanhthaopro/Desktop/Working/SW/PLA_1276_SW_inputs.csv")

In [35]:
#--------------------------------------II.1. Load The Items and PLA input file---------------------------------------------------
Items = pd.read_excel("/Users/thanhthaopro/Desktop/Working/SW/24Q4_Die_inputs.xlsx") #, sheet_name="Items"
#PLA = pd.read_csv("/Users/thanhthaopro/Desktop/Working/SW/PLA_Dot_SW_inputs.csv")
PLA = pd.DataFrame(df)

In [36]:
#Dropping External Dies from Items
Items = Items.loc[~(
    Items['DotProcess'].str.startswith('100') |
    Items['DotProcess'].str.startswith('E') |
    Items['DotProcess'].str.startswith('NA') |
    Items['DotProcess'].str.contains('nan') |
    Items['DotProcess'].str.startswith('TV')
)]

In [37]:
#II.2 Adding 'Die X (um)', 'Die Y(um)', 'PLA Dot' for Items
Items['Die X (um)'] = Items['DieX'] *1000
Items['Die Y(um)'] = Items['DieY'] *1000
#II.3 Format data types - convert SRRDieArea_mm2 from float to integer then string since str function can only concat string values
Items[["DieArea"]] = Items[["DieArea"]].astype("int")
Items[["DieArea"]] = Items[["DieArea"]].astype("str")
#Items[["PLADot"]] = Items[["PLADot"]].astype("str")
Items[["DotProcess"]] = Items[["DotProcess"]].astype("str")
Items[["FabProcess"]] = Items[["FabProcess"]].astype("str")
#Insert 'PLADotDieArea' as the column name and the value is the concatcenate of "PLADot" and "DieArea (mm2)"
#Items['DotDieArea'] = Items['PLADot'] + '_' + Items['DieArea']
#Insert 'PLADotDieArea' as the column name and the value is the concatcenate of "DotProcess" and "DieArea (mm2)"
Items['DotDieArea'] = Items['DotProcess'] + '_' + Items['DieArea']
#Insert 'PLAFabDieArea' as the column name and the value is the concatcenate of "FabProcess" and "DieArea (mm2)"
Items['FabDieArea'] = Items['FabProcess'] + '_' + Items['DieArea']
Items

,ItemID,DieArea,DieSourceItemIdentifier,CommonName,DeviceCodes,Device,ActualStreetWidthX,ActualStreetWidthY,DieX,DieY,...,WFRPCG,ProductGroup,PS_ENGINEERING_TYPE,24Q3 LaserQual,24Q4 LQ,PLADot,Die X (um),Die Y(um),DotDieArea,FabDieArea
0,34312,221,2000-083-735,Ice Lake LCC CPU Die,"8PIMCV, 8PIFCV, 8PCACV",8PCA,244.8,241.056,23.47,9.42,...,SG03,IPG,REV,DLS,DLS,1274.3,23470.0,9420.0,1274.3_221,1274_221
1,34313,477,2000-083-740,Ice Lake HCC CPU Die,8PCGCV,8PCG,489.6,562.464,23.51,20.19,...,SG08,IPG,REV,DLS,DLS,1274.3,23510.0,20190.0,1274.3_477,1274_477
2,34324,49,2000-097-962,Cedar Fork Die,88CFCV,88CF,80.0,80.000,7.00,7.00,...,S302,IPG,REV,DLS,DLS,0,7000.0,7000.0,1273.1_49,1273_49
3,34352,37,2000-109-597,CROW VALLEY,"8S10NV, 88CWCV",88CW,163.2,160.704,7.13,5.16,...,C401,IPG,REV,ULS,ULS,1274.7,7130.0,5160.0,1274.7_37,1274_37
4,34353,47,2000-109-697,Emmitsburg PCH Die,NaN,NaN,80.0,80.000,6.41,7.33,...,S305,IPG,REV,ULS,ULS,0,6410.0,7330.0,1273.1_47,1273_47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,40059,360,2000-316-425,Harvey Lake Performance Auto Stack Die,NaN,NaN,NaN,NaN,18.97,18.97,...,NaN,IPG,REV,USP,USP,1227.1,18970.0,18970.0,1227.3-1_360,1227_360
357,40064,225,2000-316-508,Arcadian Shores IO Die R,NaN,NaN,NaN,NaN,15.00,15.00,...,NaN,IPG,REV,USP,USP,1278.3-17,15000.0,15000.0,1278.3_225,1278_225
358,40065,225,2000-316-511,Arcadian Shores IO Die L,NaN,NaN,NaN,NaN,15.00,15.00,...,NaN,IPG,REV,USP,USP,1278.3-17,15000.0,15000.0,1278.3_225,1278_225
361,40078,105,2000-319-332,Titan Lake CPU 8P+16E Int Die,NaN,NaN,NaN,NaN,10.25,10.25,...,NaN,IPG,REV,USP,USP,1280,10250.0,10250.0,1280.3_105,1280_105


In [38]:
Items.dtypes

ItemID                       int64
DieArea                     object
DieSourceItemIdentifier     object
CommonName                  object
DeviceCodes                 object
Device                      object
ActualStreetWidthX         float64
ActualStreetWidthY         float64
DieX                       float64
DieY                       float64
DieZ                         int64
DotProcess                  object
FabProcess                  object
LrpDieName                  object
TransferStatus              object
WFRPkgTech                  object
WFRPCG                      object
ProductGroup                object
PS_ENGINEERING_TYPE         object
24Q3 LaserQual              object
24Q4 LQ                     object
PLADot                      object
Die X (um)                 float64
Die Y(um)                  float64
DotDieArea                  object
FabDieArea                  object
dtype: object

In [39]:
PLA.dtypes

AnalysisDate                  object
ProcessTag                    object
SRROGD_um                    float64
SRRPGD_um                    float64
SRRDeviceX_um                float64
SRRDeviceY_um                float64
SRRDieArea_mm2                object
AspectRatio                  float64
LFU                          float64
DPFOGD                         int64
DPFPGD                         int64
DPF                            int64
OGDWidthOfPGDScribe_um       float64
PGDWidthOfOGDScribe_um       float64
OGDMultOfPGDScribe            object
PGDMultOfOGDScribe            object
SingleDieSteppingOGD_um      float64
SingleDieSteppingPGD_um      float64
SingleDieSteppedArea_mm2     float64
FractureWindowOGD_um         float64
FractureWindowPGD_um         float64
GrossDPW                       int64
EdgeExclusionPrimary_mm        int64
EdgeExclusionFull_mm           int64
FrameBuildConfidenceLevel     object
PLADot                        object
PLAFab                        object
P

In [40]:
#---------------------------------------III. Get unique Items combination for the lookup-------------------------------------
#III. Remove duplicates from Items file and keep the unique combination of PLA Dot + Die X + Die Y
#Items=Items.dropna(subset=['DieArea','PLADot']) #'DieArea (mm2)'
#III.1 Remove douplicates from Items file 
Items_unique = Items.drop_duplicates(subset=['DotDieArea','Die X (um)', 'Die Y(um)'], keep='first')
Items_unique

,ItemID,DieArea,DieSourceItemIdentifier,CommonName,DeviceCodes,Device,ActualStreetWidthX,ActualStreetWidthY,DieX,DieY,...,WFRPCG,ProductGroup,PS_ENGINEERING_TYPE,24Q3 LaserQual,24Q4 LQ,PLADot,Die X (um),Die Y(um),DotDieArea,FabDieArea
0,34312,221,2000-083-735,Ice Lake LCC CPU Die,"8PIMCV, 8PIFCV, 8PCACV",8PCA,244.8,241.056,23.47,9.42,...,SG03,IPG,REV,DLS,DLS,1274.3,23470.0,9420.0,1274.3_221,1274_221
1,34313,477,2000-083-740,Ice Lake HCC CPU Die,8PCGCV,8PCG,489.6,562.464,23.51,20.19,...,SG08,IPG,REV,DLS,DLS,1274.3,23510.0,20190.0,1274.3_477,1274_477
2,34324,49,2000-097-962,Cedar Fork Die,88CFCV,88CF,80.0,80.000,7.00,7.00,...,S302,IPG,REV,DLS,DLS,0,7000.0,7000.0,1273.1_49,1273_49
3,34352,37,2000-109-597,CROW VALLEY,"8S10NV, 88CWCV",88CW,163.2,160.704,7.13,5.16,...,C401,IPG,REV,ULS,ULS,1274.7,7130.0,5160.0,1274.7_37,1274_37
4,34353,47,2000-109-697,Emmitsburg PCH Die,NaN,NaN,80.0,80.000,6.41,7.33,...,S305,IPG,REV,ULS,ULS,0,6410.0,7330.0,1273.1_47,1273_47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,40056,27,2000-316-335,Harvey Lake Mainstream GPU,NaN,NaN,NaN,NaN,5.20,5.20,...,NaN,IPG,REV,USP,USP,1278.3-17,5200.0,5200.0,1278.6_27,1278_27
355,40058,68,2000-316-397,Harvey Lake Performance GPU,NaN,NaN,NaN,NaN,8.25,8.25,...,NaN,IPG,REV,USP,USP,1278.3-17,8250.0,8250.0,1278.6_68,1278_68
356,40059,360,2000-316-425,Harvey Lake Performance Auto Stack Die,NaN,NaN,NaN,NaN,18.97,18.97,...,NaN,IPG,REV,USP,USP,1227.1,18970.0,18970.0,1227.3-1_360,1227_360
357,40064,225,2000-316-508,Arcadian Shores IO Die R,NaN,NaN,NaN,NaN,15.00,15.00,...,NaN,IPG,REV,USP,USP,1278.3-17,15000.0,15000.0,1278.3_225,1278_225


In [57]:
#--------------------------------------------------------Main Logic----------------------------------------------------------
# Function to calculate Euclidean distance
def euclidean_distance(x1, y1, x2, y2):
    return np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

# Create an empty list to store the matched rows
matched_rows = []

# Loop through each row in Items_unique
for index, item_row in Items_unique.iterrows():
    #print(f"Processing item: {item_row['DotDieArea']}, FabDieArea: {item_row['FabDieArea']}")
   
    # First, try to find matching rows based on DotDieArea
    pla_subset = PLA[PLA['PLADotDieArea'] == item_row['DotDieArea']].copy()
    #print("PLA subset for DotDieArea:", pla_subset)

    if not pla_subset.empty:
        # Calculate distances for DotDieArea matches
        pla_subset['distance'] = pla_subset.apply(
            lambda row: euclidean_distance(item_row['Die X (um)'], item_row['Die Y(um)'],
                                           row['SRROGD_um'], row['SRRPGD_um']), axis=1
        )

        min_distance = pla_subset['distance'].min()
        nearest_rows = pla_subset[pla_subset['distance'] == min_distance]
        #print("Nearest rows for DotDieArea:", nearest_rows)

        # If there are multiple nearest rows, choose the one with the largest widths
        if len(nearest_rows) > 1:
            max_ogd_width = nearest_rows['OGDWidthOfPGDScribe_um'].max()
            max_pgd_width = nearest_rows['PGDWidthOfOGDScribe_um'].max()
            pla_largest = nearest_rows[
                (nearest_rows['OGDWidthOfPGDScribe_um'] == max_ogd_width) &
                (nearest_rows['PGDWidthOfOGDScribe_um'] == max_pgd_width)
            ]
        else:
            pla_largest = nearest_rows

        # Take the first row from pla_largest if not empty
        if not pla_largest.empty:
            final_row = pla_largest.iloc[0]
            if isinstance(final_row, pd.Series):
                final_row = final_row.to_frame().T
            item_row_reset = item_row.to_frame().T.reset_index(drop=True)
            final_row_reset = final_row.reset_index(drop=True)
            combined_row = pd.concat([item_row_reset, final_row_reset], axis=1)
            matched_rows.append(combined_row)
    else:
        # If no matches for DotDieArea, check for FabDieArea
        pla_subset = PLA[PLA['PLAFabDieArea'] == item_row['FabDieArea']].copy()
        #print("PLA subset for FabDieArea:", pla_subset)

        if not pla_subset.empty:
            # Calculate distances for FabDieArea matches
            pla_subset['distance'] = pla_subset.apply(
                lambda row: euclidean_distance(item_row['Die X (um)'], item_row['Die Y(um)'],
                                               row['SRROGD_um'], row['SRRPGD_um']), axis=1
            )

            min_distance = pla_subset['distance'].min()
            nearest_rows = pla_subset[pla_subset['distance'] == min_distance]
            #print("Nearest rows for FabDieArea:", nearest_rows)

            # If there are multiple nearest rows, choose the one with the largest widths
            if len(nearest_rows) > 1:
                max_ogd_width = nearest_rows['OGDWidthOfPGDScribe_um'].max()
                max_pgd_width = nearest_rows['PGDWidthOfOGDScribe_um'].max()
                pla_largest = nearest_rows[
                    (nearest_rows['OGDWidthOfPGDScribe_um'] == max_ogd_width) &
                    (nearest_rows['PGDWidthOfOGDScribe_um'] == max_pgd_width)
                ]
            else:
                pla_largest = nearest_rows

            # Take the first row from pla_largest if not empty
            if not pla_largest.empty:
                final_row = pla_largest.iloc[0]
                if isinstance(final_row, pd.Series):
                    final_row = final_row.to_frame().T
                item_row_reset = item_row.to_frame().T.reset_index(drop=True)
                final_row_reset = final_row.reset_index(drop=True)
                combined_row = pd.concat([item_row_reset, final_row_reset], axis=1)
                matched_rows.append(combined_row)

# Concatenate all DataFrames in the matched_rows list
if matched_rows:
    matched_rows_df = pd.concat(matched_rows, ignore_index=True)
else:
    matched_rows_df = pd.DataFrame()  # Return an empty DataFrame if no matches were found

# Display the resulting DataFrame
#print("Final matched rows DataFrame:")
#print(matched_rows_df)

In [58]:
matched_rows_df

,ItemID,DieArea,DieSourceItemIdentifier,CommonName,DeviceCodes,Device,ActualStreetWidthX,ActualStreetWidthY,DieX,DieY,...,FractureWindowPGD_um,GrossDPW,EdgeExclusionPrimary_mm,EdgeExclusionFull_mm,FrameBuildConfidenceLevel,PLADot,PLAFab,PLADotDieArea,PLAFabDieArea,distance
0,34312,221,2000-083-735,Ice Lake LCC CPU Die,"8PIMCV, 8PIFCV, 8PCACV",8PCA,244.8,241.056,23.47,9.42,...,19290.34,256,3,3,NaN,1274.3,1274,1274.3_221,1274_221,131.639386
1,34313,477,2000-083-740,Ice Lake HCC CPU Die,8PCGCV,8PCG,489.6,562.464,23.51,20.19,...,20790.148,116,3,3,NaN,1274.3,1274,1274.3_477,1274_477,110.8672
2,34352,37,2000-109-597,CROW VALLEY,"8S10NV, 88CWCV",88CW,163.2,160.704,7.13,5.16,...,21620.836,1638,3,3,NaN,1274.7,1274,1274.7_37,1274_37,81.713485
3,34382,203,2000-120-901,Tiger Lake CPU-8,8PBFCV,8PBF,241.1,326.4,18.93,10.58,...,22136.548,286,3,3,NaN,1274.7,1274,1274.7_203,1274_203,140.139498
4,34385,287,2000-120-980,Snow Ridge,8PLCCV,8PLC,320.0,320.0,20.92,13.71,...,14353.54,194,3,3,NaN,1274.3,1274,1274.3_287,1274_287,80.92711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,40052,60,2000-316-296,Harvey Lake CPU,NaN,NaN,NaN,NaN,7.75,7.75,...,31798.18,983,5,5,medium,1278.3-26,1278,1278.3-26_60,1278_60,70.138038
184,40055,66,2000-316-310,Harvey Lake SCC,NaN,NaN,NaN,NaN,8.12,8.12,...,25395.22,900,4,4,NaN,1276.9-10,1276,1276.9-10_66,1276_66,82.539726
185,40056,27,2000-316-335,Harvey Lake Mainstream GPU,NaN,NaN,NaN,NaN,5.2,5.2,...,32598.82,2169,5,5,medium,1278.3-26,1278,1278.3-26_27,1278_27,1.295531
186,40058,68,2000-316-397,Harvey Lake Performance GPU,NaN,NaN,NaN,NaN,8.25,8.25,...,25695.46,866,5,5,medium,1278.3-26,1278,1278.3-26_68,1278_68,70.261409


In [59]:
#-------------------------------------------------IV. Merge back with the Items file--------------------------------------------
# Ensure 'DotDieArea' is unique in matched_rows_df to avoid merge conflicts (if necessary)
matched_rows_df_unique = matched_rows_df.drop_duplicates(subset=['DotDieArea'])

# Perform the merge (left join) based on 'DotDieArea'
Items_SW = pd.merge(
    Items,
    matched_rows_df_unique[['DotDieArea', 'SRROGD_um', 'SRRPGD_um', 'OGDWidthOfPGDScribe_um', 'PGDWidthOfOGDScribe_um'
                            ,'OGDMultOfPGDScribe','PGDMultOfOGDScribe','GrossDPW','PLADotDieArea']],  # Only the required columns from the lookup table
    how='left',  # Use 'left' join to keep all rows from Items
    left_on='DotDieArea',  # Lookup key from Items
    right_on='DotDieArea'  # Lookup key from matched_rows_df
)


In [60]:
#matched_rows_df.to_csv("/Users/thanhthaopro/Desktop/Working/SW/Merged_SW.csv")
Items_SW.to_csv("/Users/thanhthaopro/Desktop/Working/SW/24Q4_SW_outputs.csv")
